## 8.错误、调试和测试
错误（error），由于人为编写或外界干扰使得程序无法正常运行。  
调试，跟踪程序的执行，查看变量值是否正确。  
测试，确保程序输出符合我们编写的结果。

1. 错误处理
2. 调试

### 1. 错误处理
返回错误码，如 -1. 很容易混淆。
Python 内置的错误处理机制  
__try ..execpt .. finally__   

__raise__

In [12]:
try:
    print('try..')
    r = 10 / 0
    print('value: %d'%r)
    
except ZeroDivisionError  as e:
    print('Error:',e)
    
except TypeError as e:
    print('Error:',e)
    
else:
    print('No error!')
    
finally:
    print('End')
    

try..
Error: division by zero
End


可以看到，捕获异常后，程序并没有终止。  
同样是出错，但程序打印完错误信息后会继续执行，并正常退出

### 调用栈
通过调用栈检测异常
```python
Traceback (most recent call last):
  File "err.py", line 11, in <module>
    main()
  File "err.py", line 9, in main
    bar('0')
  File "err.py", line 6, in bar
    return foo(s) * 2
  File "err.py", line 3, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero
```
可以清楚分析到，问题出在err.py的第三行，除以0了

### 记录错误 
调用 logging函数

### 抛出错误
__raise__ 指令  
Python的内置函数会抛出很多类型的错误，我们自己编写的函数也可以抛出错误。

In [13]:
# err_raise.py
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

foo('0')

FooError: invalid value: 0

In [16]:
# err_reraise.py

def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise
try:
    bar()

except ValueError as e:
    print('I got this Error,',e)

ValueError!
I got this Error, invalid value: 0


在bar()函数中，我们明明已经捕获了错误，但是，打印一个ValueError!后，又把错误通过raise语句抛出去了，这不有病么？

其实这种错误处理方式不但没病，而且相当常见。捕获错误目的只是记录一下，便于后续追踪。

由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。

### 练习
运行下面的代码，根据异常信息进行分析，定位出错误源头，并修复：

In [41]:
from functools import reduce

def str2num(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError as e:
            print("Error:",e)
            print('Transfer %r to zero'%s)
            return 0
            
    
def calc(exp):
    ss = exp.split('+')
    ns = map(str2num, ss)
    return reduce(lambda acc, x: acc + x, ns)

def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6')
    print('99 + 88 + 7.6 =', r)
    r = calc('99 + 88 +a')
    print('99 + 88 + a =', r)

main()

100 + 200 + 345 = 645
99 + 88 + 7.6 = 194.6
Error: could not convert string to float: 'a'
Transfer 'a' to zero
99 + 88 + a = 187


### 2. 调试
1. 无脑 print()
2. 断言 assert
>相当于判断，assert A,B；判断A，不正确就输出B.  
3. 日志 logging() 效果表示不出来
4. 命令端的调试 -pdb




In [53]:
import logging
logging.basicConfig(level=logging.INFO)

def foo(s):
    n = int(s)
    # print('>>> n = %d' % n)
    # assert n != 0, '>>> n = %d' % n
    logging.info('n = %d' %n)
    return 10 / n

def main():
    try:
        foo('0')
    except ZeroDivisionError:
        print('I got the error.')

main()

I got the error.


启动Python解释器时可以用-O参数来关闭assert，是大写英文 -O  
python -O train.py

#### pdb
python -m pdb train.py  
启动后，pdb定位到下一步要执行的代码  -> s = '0'  
```
注意指令 
l --（list）显示所有代码  
n --（next）执行下一步  
p 变量名 --查看变量的值  
q --（quiet）退出  
```
pdb.set_trace()在代码中加入指令，相当于pdb模型下的断点